In [1]:
import sys
import os

# Add the path to the parent module
sys.path.append(os.path.abspath('../..'))

import warnings
warnings.filterwarnings('ignore')


import deeppy as dp

import torch
import torch.optim as optim
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


import numpy as np
import matplotlib.pyplot as plt

import tiktoken



from deeppy import LearnFrame,LayerGenerator,FromLoader
from deeppy.models.nlp.gpt import GPT
from deeppy import GPTText


In [2]:
encoding = tiktoken.encoding_for_model("gpt-2")


vocab_size = encoding.n_vocab
context_size = 8

In [3]:
with open("assets/shakespeare.txt", "r", encoding = "utf-8") as f:
    text = f.read()

data = GPTText(text=text, tokenizer=encoding, context_size = context_size)

In [4]:
Optimizer_params = {
    "optimizer":optim.AdamW,
    "optimizer_args":{"lr":5e-4, "amsgrad" : True, "weight_decay" : 0, "betas" : (0.9, 0.95)},
    "clipper":nn.utils.clip_grad_norm_,
    "clipper_params":{"max_norm" : 1.0},
    "scheduler_params":None,
}

GPT_params = {
    "optimizer_params":Optimizer_params,
    "vocab_size":vocab_size,
    "embed_dim":256,
    "num_heads":5,
    "num_layers":5,
    "context_size":context_size,
    "device":device,
    "criterion":nn.CrossEntropyLoss(ignore_index = -1),
}

model = GPT(**GPT_params)

print(f"Total parameters : {sum(p.numel() for p in model.net.parameters()) / 1e6} Million")

Total parameters : 3.267584 Million


In [5]:
lf = LearnFrame(model,data)

In [108]:
epoch = 5000

for i in range(epoch):
    lf.optimize()
    lf.test()

    if i % 100 == 0:
        lf.plot(show_result=False, log=True)
lf.plot(show_result=True, log=True, save = "GPT.png")

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [106]:
txt = "On this very beautiful day, let us"
for i in range(10):
    inp = torch.tensor(encoding.encode(txt)).unsqueeze(0)
    out = model.generate(inp, 9).cpu()
    inp = out
    txt += encoding.decode(out.squeeze(0).tolist())
print(txt)

On this very beautiful day, let us.


ABKEES:
And,,


L:

I:
I
I

, the,
I
I,
I
I,
And,
I,

IUS:
IUS,
I, the the,, the,,
And the,

And:
I,:
I,,,,,,,
I the,
